# Advent of Code 2022

## Day 7: No Space Left On Device

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

The difficulty went back up compared to yesterday!

I built three classes `Node` and its subclasses `Folder` and `File`.

`Node` tracks the name of a node on the file system and its parent (I could probably solve this without linking the parent in the node, but this made it easier). `Folder` tracks child nodes (or `None` if an `ls` command has not been issued yet), `File` tracks the size.

Some of the methods like `full_name` are just there for testing and debugging. The important work is done by `compute_size` which is recursive on `Folder`.

`process_commands` does the work of looping over commands and building the tree, tracking `cwd` and navigating up and down. Once this process is complete I had a recursive `get_all_folders` mothod to get a list of folders and then it's relatively simple to apply the logic described in the puzzle for deciding on the correct folder by size. Some extra print lines were in part 2 to check the maths of file sizes was working.

In [ ]:
from typing import Optional, Iterator
from abc import ABC

In [ ]:
def read(filename: str, limit: Optional[int] = None) -> Iterator[str]:
    with open(filename) as file:
        count = 0
        for line in file:
            line = line.strip()
            yield line
            count += 1
            if limit is not None and count >= limit:
                break

In [ ]:
class Node(ABC):

    __slots__ = ['parent', 'name']

    def __init__(self, parent: Optional['Node'], name: str):
        self.parent = parent
        self.name = name

    def __str__(self):
        return self.to_string()

    def __repr__(self):
        return str(self)

    def to_string(self, indent: int = 0):
        pass

    def full_name(self):
        if self.parent is None:
            return self.name
        elif self.parent.parent is None:
            return self.parent.name + self.name
        else:
            return self.parent.full_name() + '/' + self.name

    def compute_size(self):
        pass


class Folder(Node):

    __slots__ = [ 'children']

    def __init__(self, parent: Optional['Node'], name: str, children: Optional[list[Node]] = None):
        super().__init__(parent, name)
        self.children = children

    def to_string(self, indent: int = 0):
        rv = ' ' * (4 * indent) + '- ' + self.name + ' (dir)\n'
        if self.children is None:
            rv += ' ' * (4 * (indent + 1)) + 'ERROR - UNKNOWN CONTENTS!!!\n'
        else:
            for c in self.children:
                rv += c.to_string(indent + 1)
        return rv

    def compute_size(self):
        return sum([c.compute_size() for c in self.children])

class File(Node):

    __slots__ = ['size']

    def __init__(self, parent: Optional['Node'], name: str, size: int = None):
        super().__init__(parent, name)
        self.size = size

    def to_string(self, indent: int = 0):
        return ' ' * (4 * indent) + '- ' + self.name + ' (file, size=' + str(self.size) + ')\n'

    def compute_size(self):
        return self.size

In [ ]:
def read_commands(filename: str) -> Iterator[tuple[str, list[str]]]:

    command, output = None, []

    for line in read(filename):

        # line is a command
        if line.startswith('$ '):

            if command is not None:
                yield command, output

            command, output = line[2:], []

        # line is an output
        else:

            assert command is not None
            output.append(line)

    if command is not None:
        yield command, output

In [ ]:
def process_commands(file):

    root = Folder(None, '/')
    cwd = None

    for command, output in read_commands(file):

        if cwd is None:
            assert (command == 'cd /'), f'first command was {command} instead of cd /'

        # change directory (absolute)
        if command == 'cd /':
            cwd = root

        # up one directory
        elif command == 'cd ..':
            cwd = cwd.parent

        # go into a directory
        elif command.startswith('cd'):
            _, name = command.split(' ')
            child = None
            for c in cwd.children:
                if c.name == name:
                    child = c
            assert child is not None
            cwd = child

        # list directory contents
        elif command == 'ls':
            assert type(cwd) == Folder
            assert (cwd.children is None), f'{cwd.full_name()}, already has contents, but trying to list again'
            cwd.children = []
            for o in output:
                size, name = o.split(' ')
                if size == 'dir':
                    new = Folder(cwd, name)
                    cwd.children.append(new)
                else:
                    size = int(size)
                    new = File(cwd, name, size)
                    cwd.children.append(new)

        # error
        else:
            raise ValueError(f'unknown command: {command}')

    return root

In [ ]:
def get_all_folders(root: Optional[Folder], into: list[Folder]) -> None:
    if type(root) == Folder:
        into.append(root)
        for c in root.children:
            get_all_folders(c, into)

In [ ]:
INPUT_FILE = 'data/input07.txt'

### Part 1

In [ ]:
MAX_SIZE = 100000

def main():
    final_structure = process_commands(INPUT_FILE)
    folders = []
    get_all_folders(final_structure, folders)
    total = 0
    for f in folders:
        if f.compute_size() <= MAX_SIZE:
            total += f.compute_size()

    # print(final_structure)  # for testing
    # print()
    print('total size =', final_structure.compute_size())
    print(f'sum of folders at most {MAX_SIZE} = {total}')

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
FILE_SYSTEM_SIZE = 70000000
UPDATE_SIZE = 30000000

def main():
    final_structure = process_commands(INPUT_FILE)
    folders = []
    get_all_folders(final_structure, folders)
    possible_folder = []
    current_used = final_structure.compute_size()
    print('current space used :', current_used)
    print('needed for update :', UPDATE_SIZE)
    print('file system size :', FILE_SYSTEM_SIZE)
    print('need to free at least :', (UPDATE_SIZE + current_used - FILE_SYSTEM_SIZE))
    for f in folders:
        possible_folder.append((f.compute_size(), f.full_name()))
    possible_folder.sort()
    for size, folder in possible_folder:
        if size >= UPDATE_SIZE + current_used - FILE_SYSTEM_SIZE:
            print(f'folder to delete is {folder}, of size {size}')
            return
    print('nothing found!')

In [ ]:
if __name__ == '__main__':
    main()